# Solarlog Data Analyse
This Kernal gives an overview about the PV-Data over all sourced plants.

It takes about 2 min to run the whole notebook.

## Import

More information about impyla see here: https://github.com/cloudera/impyla.

In [1]:
from impala.dbapi import connect 
from impala.util import as_pandas
import pandas as pd # For Data Handling
import numpy as np  # data processing, CSV file I/O (e.g. pd.read_csv)
import math as mt
import datetime as dt # To convert timestamp data
import matplotlib   #for plots
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(color_codes=True)
matplotlib.style.use('ggplot')
%matplotlib inline

## Load Data from HDFS with Impala

Die folgende querry benötigt 91s

In [2]:
conn = connect(host='172.16.10.102', port=21050)
cur = conn.cursor()
cur.execute('refresh solarlog_roh')
cur.execute('SELECT `timestamp`,cur_yield_watt,plantname FROM solarlog_roh order by `timestamp` limit 150000')
dfp = as_pandas(cur)
cur.close()
conn.close()
dfp.head()

TTransportException: TTransportException(message="Could not connect to ('172.16.10.102', 21050)", type=1)

In [ ]:
len(dfp)

In [ ]:
dfp.info()

In [ ]:
dfp.describe()

## Daten Verstehen

In [ ]:
# Change epochtime from timestamp to date time
df = pd.DataFrame() # New Data Frame
# Convert to datetime and add 1h corret time shift by collecting data
df['datetime'] = dfp.timestamp.apply(dt.datetime.utcfromtimestamp) + pd.Timedelta(hours=1)
df.sort_values(['datetime'])
df['value_watt'] = dfp.cur_yield_watt
df['name'] = dfp.plantname
df.head()

In [ ]:
df.name.value_counts()

In [ ]:
df.info()

In [ ]:
df.groupby('name').describe().value_watt
dfp1 = df.copy()

In [ ]:
df1 = dfp1[dfp1['name'] == 'Pfadiheim Baar']
df2 = dfp1[dfp1['name'] == 'Meyer Rolf, Baar']
df3 = dfp1[dfp1['name'] == 'Winsun AG, Steg']
df4 = dfp1[dfp1['name'] == 'Himmelrichstrasse 1ab, Baar']
df5 = dfp1[dfp1['name'] == 'Betschart René, Steinhausen']
df6 = dfp1[dfp1['name'] == 'Loosmann Johannes, Hausen am Albis']
df7 = dfp1[dfp1['name'] == 'Haller Roman, Rifferswil']
#df8 = dfp1[dfp1['name'] == 'Maechler Paul, Allenwinden']

In [ ]:
# Time Series Plot
plt.figure(figsize = (15,8))
plt.plot(dfp1.datetime,dfp1.value_watt)
plt.ylabel("Leistung [W]",size=14)
plt.xlabel("Time",size=14)
plt.title("Pfadiheim, Baar")
plt.show()

In [ ]:
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
init_notebook_mode(connected=True) #need to use plotly offline

data = Data([Scatter(x=df1.datetime, y=df1.value_watt, name='Pfadiheim Baar'),
             Scatter(x=df2.datetime, y=df2.value_watt, name='Meyer Rolf, Baar'),
             Scatter(x=df3.datetime, y=df3.value_watt, name='Winsun AG, Steg'),
             Scatter(x=df4.datetime, y=df4.value_watt, name='Himmelrichstrasse 1ab, Baar'),
             Scatter(x=df5.datetime, y=df5.value_watt, name='Betschart René, Steinhausen'),
             Scatter(x=df6.datetime, y=df6.value_watt, name='Loosmann Johannes, Hausen am Albis'),
             Scatter(x=df7.datetime, y=df7.value_watt, name='Haller Roman, Rifferswil')])

layout = Layout(
    title='Leistung aller Anlagen',
    yaxis=dict(title='Leistung in [kW]'),
    xaxis=dict(title='Zeit')
)

fig = Figure(data=data, layout=layout)

iplot(fig, filename = 'basic-line')

## Lade Solar Inventar Tabelle
Die Daten der Winsun Anlage in der inverntar_solar Tabelle werden direkt über die Solarlog API in HDFS konsumiert.

In [ ]:
conn = connect(host='172.16.10.102', port=21050)
cur = conn.cursor()
cur.execute('refresh inventory_solar')
cur.execute('SELECT * FROM inventory_solar')
dfis = as_pandas(cur)
cur.close()
conn.close()
#To see all tables tip only dfp without .head()
dfis.head(1)

In [ ]:
df = dfis.set_index('index')
df['nennleistung'] = df.anlagenleistung
df['name'] = df.plantname
df['ort'] = df.zip
df.drop('anlagenleistung', axis=1, inplace=True)
df.drop('plantname', axis=1, inplace=True)
df.drop('zip', axis=1, inplace=True)
df

## Vergleich der Anlagen

In [ ]:
df[df.name == 'Pfadiheim Baar'].nennleistung*1000

In [ ]:
dfp1[dfp1.name=='Pfadiheim Baar'].value_watt.head()

In [ ]:
len(dfp1[dfp1.name=='Pfadiheim Baar'].value_watt/(int(df[df.name == 'Pfadiheim Baar'].nennleistung)))

In [ ]:
# Um die Anlagen Vergelichbar zu machen wird nun der spezifische Ertrag berechnet
dfp1['spez_leistung'] = dfp1.value_watt
dfp1.loc[dfp1[dfp1.name=='Pfadiheim Baar'].index,"spez_leistung"] = dfp1[dfp1.name=='Pfadiheim Baar'].value_watt/(int(df[df.name == 'Pfadiheim Baar'].nennleistung*1000))
dfp1.loc[dfp1[dfp1.name=='Winsun AG, Steg'].index,"spez_leistung"] = dfp1[dfp1.name=='Winsun AG, Steg'].value_watt/(int(df[df.name == 'Winsun AG, Steg'].nennleistung*1000))
dfp1.loc[dfp1[dfp1.name=='Meyer Rolf, Baar'].index,"spez_leistung"]= dfp1[dfp1.name=='Meyer Rolf, Baar'].value_watt/(int(df[df.name == 'Meyer Rolf, Baar'].nennleistung*1000))
dfp1.loc[dfp1[dfp1.name=='Himmelrichstrasse 1ab, Baar'].index,"spez_leistung"]= dfp1[dfp1.name=='Himmelrichstrasse 1ab, Baar'].value_watt/(int(df[df.name == 'Himmelrichstrasse 1ab, Baar'].nennleistung*1000))
dfp1.loc[dfp1[dfp1.name=='Betschart René, Steinhausen'].index,"spez_leistung"]= dfp1[dfp1.name=='Betschart René, Steinhausen'].value_watt/(int(df[df.name == 'Betschart René, Steinhausen'].nennleistung*1000))
dfp1.loc[dfp1[dfp1.name=='Loosmann Johannes, Hausen am Albis'].index,"spez_leistung"]= dfp1[dfp1.name=='Loosmann Johannes, Hausen am Albis'].value_watt/(int(df[df.name == 'Loosmann Johannes, Hausen am Albis'].nennleistung*1000))
dfp1.loc[dfp1[dfp1.name=='Haller Roman, Rifferswil'].index,"spez_leistung"]= dfp1[dfp1.name=='Haller Roman, Rifferswil'].value_watt/(int(df[df.name == 'Haller Roman, Rifferswil'].nennleistung*1000))
dfp1.head()                                                   

In [ ]:
dfp1[dfp1.name=='Himmelrichstrasse 1ab, Baar'].head()

In [ ]:
dfp1.fillna(0)
dfp1.describe()

In [ ]:
data2 = Data([Scatter(x=dfp1[dfp1.name == 'Pfadiheim Baar'].datetime, y=dfp1[dfp1.name == 'Pfadiheim Baar'].spez_leistung, name='Pfadiheim, Baar'),
             Scatter(x=dfp1[dfp1.name == 'Meyer Rolf, Baar'].datetime, y=dfp1[dfp1.name == 'Meyer Rolf, Baar'].spez_leistung, name='Meyer, Baar'),
             Scatter(x=dfp1[dfp1.name == 'Betschart René, Steinhausen'].datetime, y=dfp1[dfp1.name == 'Betschart René, Steinhausen'].spez_leistung, name='Betschart, Steinhausen'),
             Scatter(x=dfp1[dfp1.name == 'Himmelrichstrasse 1ab, Baar'].datetime, y=dfp1[dfp1.name == 'Himmelrichstrasse 1ab, Baar'].spez_leistung, name='Himmelrichstrasse, Baar'),
             Scatter(x=dfp1[dfp1.name == 'Loosmann Johannes, Hausen am Albis'].datetime, y=dfp1[dfp1.name == 'Loosmann Johannes, Hausen am Albis'].spez_leistung, name='Loosmann, Hausen am Albis'),
             Scatter(x=dfp1[dfp1.name == 'Haller Roman, Rifferswil'].datetime, y=dfp1[dfp1.name == 'Haller Roman, Rifferswil'].spez_leistung, name='Haller Rifferswil'),
             Scatter(x=dfp1[dfp1.name == 'Winsun AG, Steg'].datetime, y=dfp1[dfp1.name == 'Winsun AG, Steg'].spez_leistung, name='Winsun AG, Steg')])

layout2 = Layout(
    title='Spezifische Leistung aller Anlagen',
    yaxis=dict(title='Spezifische Leistung [kW/kWp]'),
    xaxis=dict(title='Zeit')
)

fig2 = Figure(data=data2, layout=layout2)

iplot(fig2, filename = 'basic-line')

In [ ]:
#Zeit Beschränkung
today = dt.date.today()
week_ago = today - dt.timedelta(days=30)
dfpt = dfp1[dfp1.datetime > week_ago]
#Prepare Plot
data2 = Data([Scatter(x=dfpt[dfpt.name == 'Pfadiheim Baar'].datetime, y=dfpt[dfpt.name == 'Pfadiheim Baar'].spez_leistung, name='Pfadiheim, Baar'),
             Scatter(x=dfpt[dfpt.name == 'Meyer Rolf, Baar'].datetime, y=dfpt[dfpt.name == 'Meyer Rolf, Baar'].spez_leistung, name='Meyer, Baar'),
             Scatter(x=dfpt[dfpt.name == 'Betschart René, Steinhausen'].datetime, y=dfpt[dfpt.name == 'Betschart René, Steinhausen'].spez_leistung, name='Betschart, Steinhausen'),
             Scatter(x=dfpt[dfpt.name == 'Himmelrichstrasse 1ab, Baar'].datetime, y=dfpt[dfpt.name == 'Himmelrichstrasse 1ab, Baar'].spez_leistung, name='Himmelrichstrasse, Baar'),
             Scatter(x=dfpt[dfpt.name == 'Loosmann Johannes, Hausen am Albis'].datetime, y=dfpt[dfpt.name == 'Loosmann Johannes, Hausen am Albis'].spez_leistung, name='Loosmann, Hausen am Albis'),
             Scatter(x=dfpt[dfpt.name == 'Haller Roman, Rifferswil'].datetime, y=dfpt[dfpt.name == 'Haller Roman, Rifferswil'].spez_leistung, name='Haller Rifferswil'),
             Scatter(x=dfpt[dfpt.name == 'Winsun AG, Steg'].datetime, y=dfpt[dfpt.name == 'Winsun AG, Steg'].spez_leistung, name='Winsun AG, Steg')])

layout2 = Layout(
    title='Spezifische Leistung aller Anlagen',
    yaxis=dict(title='Spezifische Leistung [kW/kWp]'),
    xaxis=dict(title='Zeit')
)

fig2 = Figure(data=data2, layout=layout2)

iplot(fig2, filename = 'basic-line')

In [ ]:
dfsp = pd.DataFrame(index=dfp1.datetime)
#df3 = pd.DataFrame(df2['teams'].values.tolist(), columns=['team1','team2'])
dfsp = pd.DataFrame(dfsp['name'].,colums=['']

dfsp['Pfadiheim Baar'] = dfp1[dfp1.name == 'Pfadiheim Baar'].spez_leistung.value
dfsp['Meyer Rolf, Baar'] = dfp1[dfp1.name == 'Meyer Rolf, Baar'].spez_leistung
dfsp.head()

In [ ]:
dfsp.corr()